In [2]:
import requests
from datetime import date
import sys
import statsapi
from pprint import pprint
import json
import os
from google.cloud import bigquery


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.expanduser("~/Documents/jupyter/MLB/credentials/mlb-analysis-463501-869e729270f3.json")

client = bigquery.Client()
print("Project:", client.project)

Project: mlb-analysis-463501


In [38]:
# # Get today's schedule
today = date.today().isoformat() 
schedule = statsapi.schedule(start_date=today, end_date=today, sportId=1, leagueId=104)

mlb = 104
currentSeason = 2025

#Pull only attribute in list into new array
gamePks = [game['game_id'] for game in schedule]
print(gamePks)

[777272, 777284, 777265, 777273, 777290, 777289, 777276, 777294, 777275, 777277, 777279, 777266, 777269, 777270, 777267]


In [54]:
#Helper function for team stats by season
def team_stat_data(team, group, season, stats='season'):
    url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/stats"
    params = {
        "season": season,
        "group": group,
        "stats": stats
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json()["stats"][0]["splits"][0]

In [59]:
#Create table for todays games (primary key gamePk)
games = []
teams = []
team_stats = []
players = []
player_game_stats = []
for game_pk in gamePks:
    game = statsapi.get("game", {"gamePk": game_pk})
    gamedata = game['gameData']
    games.append({
        "gamePk": game_pk,
        "datetime": gamedata['datetime']['dateTime'],
        "venue_id": gamedata['venue']['id'],
        "status": game['gameData']['status']['detailedState'],
        "weather": game.get("weather", {}).get("condition"),
        "doubleHeader": gamedata['game']['doubleHeader']
    })
    for side in ["home", "away"]:
        team = gamedata["teams"][side]
        team_id = team["id"]
        hitting_stats = team_stat_data(team = team_id, group = "hitting", season = currentSeason)["stat"]
        pitching_stats = team_stat_data(team = team_id, group = "pitching", season = currentSeason)["stat"]
        fielding_stats = team_stat_data(team = team_id, group = "fielding", season = currentSeason)["stat"]
        hitting_stats["type"] = "hitting"
        pitching_stats["type"] = "pitching"
        fielding_stats["type"] = "fielding"
        teams.append({
            "team_id": team["id"],
            "name": team["name"],
            "side": side,
            "gamePk": game_pk,
            "league": team["league"],
            "division": team["division"]
        })
        team_stats.append({
            "team_id": team["id"],
            "season": currentSeason
        })
        team_stats.append(hitting_stats)
        team_stats.append(pitching_stats)
        team_stats.append(fielding_stats)
        pprint(team_stats)
        sys.exit()
        time.sleep(2)
pprint(teams)

AttributeError: 'dict' object has no attribute 'append'